In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.metrics.aline import similarity_matrix
from pandas import value_counts
from win32evtlogutil import langid

- kaggle dataset (초안)

In [51]:
book_df=pd.read_csv('book_review_data.csv')

In [52]:
# 1) 필요한 컬럼 도출
# 2) 형변환 (float > int)

book_df=book_df[['user_id',"age","rating","book_title",'book_author','year_of_publication','img_m','Summary','Category','country','Language']]
book_df[['age',"year_of_publication"]]=book_df[['age',"year_of_publication"]].astype(int)

In [53]:
# 최소 평가 개수 ( 평가가 30개보다 많은 책만 추출 )
counts=book_df['book_title'].value_counts()
title = counts[counts > 30].index
book_filter_df=book_df[book_df['book_title'].isin(title)]

In [54]:
# 최소 평가 개수 ( 평가가 20개보다 많은 사용자만 추출 )
counts=book_filter_df["user_id"].value_counts()
user_id_=counts[counts>10].index
book_filter_df = book_filter_df[book_filter_df['user_id'].isin(user_id_)]

In [55]:
# 무의미한 0점 평가 제거
rating_zero=book_filter_df[book_filter_df['rating']==0].index
book_filter_df.drop(rating_zero, axis=0, inplace=True)

In [56]:
# 한 카테고리안에 책이 5권 미만인 경우 제거
counts=book_filter_df["Category"].value_counts()
categories = counts[counts > 5].index
book_filter_df = book_filter_df[book_filter_df['Category'].isin(categories)]
idx=book_filter_df[book_filter_df["Summary"]==9].index
book_filter_df.drop(idx,axis=0,inplace=True)

In [57]:
book_filter_df.shape
len(book_filter_df['user_id'].unique())

4920

In [58]:
user_book_df=book_filter_df.pivot_table(index='user_id',columns='book_title',values='rating',fill_value=0)
user_book_df.shape # user간 유사도 구할거 아니니까
book_user_df=user_book_df.T

In [59]:
user_book_df.iloc[55].sort_values(ascending=False)[:10]

book_title
It                                                                          8.0
Buddha of Suburbia                                                          8.0
Christine                                                                   8.0
Thinner                                                                     8.0
Firestarter (Signet Book)                                                   7.0
Practical Demonkeeping                                                      7.0
Dolores Claiborne                                                           7.0
Something Wicked This Way Comes                                             6.0
Cujo                                                                        6.0
Stupid White Men ...and Other Sorry Excuses for the State of the Nation!    6.0
Name: 4334, dtype: float64

In [60]:
from sklearn.metrics.pairwise import cosine_similarity
book_similarity=cosine_similarity(book_user_df,book_user_df)
book_similarity_df=pd.DataFrame(book_similarity)

In [61]:
book_filter_df["age"].value_counts()

age
34    25722
33     2853
29     2631
28     2565
32     2453
      ...  
11        8
76        7
84        6
10        5
97        4
Name: count, Length: 78, dtype: int64

In [85]:
def make_ages(age):
    if 10<=age<20:
        return 10
    elif 20<=age<30:
        return 20
    elif 30<=age<40:
        return 30
    elif 40<=age<50:
        return 40
    elif 50<=age<60:
        return 50
    elif 60<=age<70:
        return 60
    elif 70<=age<80:
        return 70
    else:
        return 80

In [86]:
book_filter_df['ages'] = book_filter_df['age'].apply(make_ages)

In [106]:
book_filter_df["book_title"]=book_filter_df['book_title'].str.lower()
book_filter_df

,user_id,age,rating,book_title,book_author,year_of_publication,img_m,Summary,Category,country,Language,ages
31,11676,34,9,the kitchen god's wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,A Chinese immigrant who is convinced she is dy...,['Fiction'],NaN,en,30
32,29526,26,9,the kitchen god's wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,A Chinese immigrant who is convinced she is dy...,['Fiction'],usa,en,20
34,46398,37,9,the kitchen god's wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,A Chinese immigrant who is convinced she is dy...,['Fiction'],usa,en,30
36,49635,34,9,the kitchen god's wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,A Chinese immigrant who is convinced she is dy...,['Fiction'],usa,en,30
42,148712,34,10,the kitchen god's wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,A Chinese immigrant who is convinced she is dy...,['Fiction'],usa,en,30
...,...,...,...,...,...,...,...,...,...,...,...,...
1026938,270820,52,7,the vision,Dean R. Koontz,1986,http://images.amazon.com/images/P/0606298347.0...,9,9,usa,9,50
1027926,273197,55,8,island,Thomas Perry,1989,http://images.amazon.com/images/P/0380706636.0...,"A husband-and-wife con team, a soldier of fort...",['Fiction'],usa,en,50
1028086,273718,32,6,pearl,Anon,1999,http://images.amazon.com/images/P/1853266094.0...,9,9,united kingdom,9,30
1029323,275970,46,9,me talk pretty one day,David Sedaris,2001,http://images.amazon.com/images/P/1586210661.0...,9,9,usa,9,40


In [141]:
ages_book_=book_filter_df.groupby(['ages','book_title']).size().reset_index(name='count')

,ages,book_title,count
0,10,'salem's lot,1
1,10,16 lighthouse road,1
2,10,1984,5
3,10,1st to die: a novel,1
4,10,3001: the final odyssey,1
...,...,...,...
17214,80,wayside school gets a little stranger (wayside school),1
17215,80,what to expect the first year,1
17216,80,wild horses,1
17217,80,writing down the bones,1


In [142]:
ages_book_df=ages_book_.pivot_table(index='ages',columns='book_title',values='count',fill_value=0)
book_ages_df=ages_book_df.T
book_ages_df[:5]

ages,10,20,30,40,50,60,70,80
book_title,,,,,,,,
'salem's lot,1.0,5.0,6.0,1.0,2.0,0.0,0.0,0.0
10 lb. penalty,0.0,1.0,5.0,4.0,1.0,1.0,1.0,0.0
101 dalmatians,0.0,3.0,2.0,2.0,1.0,0.0,0.0,0.0
16 lighthouse road,1.0,1.0,6.0,3.0,2.0,0.0,0.0,0.0
1984,5.0,28.0,24.0,12.0,1.0,0.0,0.0,0.0


In [143]:
from sklearn.metrics.pairwise import cosine_similarity
ages_similarity=cosine_similarity(book_ages_df,book_ages_df)
ages_similarity_df=pd.DataFrame(ages_similarity)